In [2]:
import os
import pandas as pd
import numpy as np

In [3]:
cn_dict_with_images = np.load("data/cn_dict_with_images.npy",allow_pickle='TRUE').item()
ad_dict_with_images = np.load("data/ad_dict_with_images.npy",allow_pickle='TRUE').item()
smci_dict_with_images = np.load("data/smci_dict_with_images.npy",allow_pickle='TRUE').item()
pmci_dict_with_images = np.load("data/pmci_dict_with_images.npy",allow_pickle='TRUE').item()